In [2]:
import warnings
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

logger.enable("edgedroid")

data = default_data.load_curve_fitting_data()
data

,participant,prev_ttf,exec_time,prev_duration,neuro,neuroticism
0,134146,0.0,3.029,"[-inf, 5.0)","[-inf, 0.5)",0.375
1,134146,0.0,4.004,"[-inf, 5.0)","[-inf, 0.5)",0.375
2,134146,0.0,2.625,"[-inf, 5.0)","[-inf, 0.5)",0.375
3,134146,0.0,4.680,"[-inf, 5.0)","[-inf, 0.5)",0.375
4,134146,0.0,5.197,"[-inf, 5.0)","[-inf, 0.5)",0.375
...,...,...,...,...,...,...
6755,137353,0.0,6.358,"[5.0, 9.0)","[0.5, inf)",0.625
6756,137353,0.0,3.840,"[5.0, 9.0)","[0.5, inf)",0.625
6757,137353,0.0,2.932,"[9.0, inf)","[0.5, inf)",0.625
6758,137353,0.0,1.874,"[9.0, inf)","[0.5, inf)",0.625


In [7]:
import numpy as np

data["outlier"] = False
for _, df in data.groupby(["prev_ttf", "prev_duration"]):
    outliers = ((df["exec_time"] > np.percentile(df["exec_time"], 95)) | (df["exec_time"] < np.percentile(df["exec_time"], 5)))
    outlier_idx = df.loc[outliers].index
    data.loc[outlier_idx, "outlier"] = True
data

,participant,prev_ttf,exec_time,prev_duration,neuro,neuroticism,outlier
0,134146,0.0,3.029,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
1,134146,0.0,4.004,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
2,134146,0.0,2.625,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
3,134146,0.0,4.680,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
4,134146,0.0,5.197,"[-inf, 5.0)","[-inf, 0.5)",0.375,False
...,...,...,...,...,...,...,...
6755,137353,0.0,6.358,"[5.0, 9.0)","[0.5, inf)",0.625,False
6756,137353,0.0,3.840,"[5.0, 9.0)","[0.5, inf)",0.625,False
6757,137353,0.0,2.932,"[9.0, inf)","[0.5, inf)",0.625,False
6758,137353,0.0,1.874,"[9.0, inf)","[0.5, inf)",0.625,False


In [12]:
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm

result_rows = deque()
for participant in tqdm(data["participant"].unique(), desc="Run"):
    class MultiCurveFittingTestModel(MultiCurveFittingExecutionTimeModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
    class PowerFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (PowerFit,)
        
    class SquareFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (SquareFit,)
        
    class CubeFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (CubeFit,)
        
    class ExponentialFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (ExponentialFit,)
        
    def get_test_data(*args, **kwargs) -> (
        Tuple[
            pd.DataFrame,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
        ]
    ):
        data, *rest = ExecutionTimeModel.get_data()
        return (data[data["run_id"] != participant].copy(), *rest)

    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
        
    class NaiveAggTestModel(FirstOrderAggregateETM):
        get_data = get_test_data
            
    run_data = data[data["participant"] == participant].copy()

    neuroticism = run_data["neuroticism"].unique()[0]
    naive_model = NaiveTestModel()
    naive_mean_model = NaiveAggTestModel(np.mean)
    naive_median_model = NaiveAggTestModel(np.median)
    
    power_fit_model = PowerFitTestModel(neuroticism)
    square_fit_model = SquareFitTestModel(neuroticism)
    cube_fit_model = CubeFitTestModel(neuroticism)
    exponential_fit_model = ExponentialFitTestModel(neuroticism)
    multi_fit_model = MultiCurveFittingExecutionTimeModel(neuroticism)
    
    # deque containing (model name, model object, ttf_bins, window_size, kernel name)
    models: Deque[Tuple[str, ExecutionTimeModel, int, int, str]] = deque()
    models.extend((
        ("1st order", naive_model, 0, 0, "none"),
        ("1st order (median)", naive_median_model, 0, 0, "none"),
        ("1st order (mean)", naive_mean_model, 0, 0, "none"),
        ("a * x^b + c", power_fit_model , 0, 0, "none"),
        ("a * x^2 + b * x + c", square_fit_model, 0, 0, "none"),
        ("a * x^3 + b * x^2 + c * x + d", cube_fit_model, 0, 0, "none"),
        ("a * e^x + b", exponential_fit_model, 0, 0, "none"),
        ("multi curve", multi_fit_model, 0, 0, "none")
    ))
        
    for _ in range(1):
        for i, (prev_ttf, exec_time, neuro, prev_duration, outlier) in enumerate(run_data[["prev_ttf", "exec_time", "neuro", "prev_duration", "outlier"]].itertuples(index=False)):
            for model_name, model, ttf_bins, window_size, kernel_name in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)
                    
                prediction = model.get_execution_time()
                error = np.nan if outlier else (prediction - exec_time)
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": error,
                    "sqr_error": np.square(error),
                    "model": model_name,
                    "duration": prev_duration,
                    "neuro": neuro,
                    # "kernel": kernel_name,
                    # "ttf_bins": ttf_bins,
                    # "window_size": window_size,
                })

results = pd.DataFrame(result_rows)
for col in ("model", ):  # "kernel"):
    results[col] = results[col].astype("category")

results.to_parquet("./full_validation_ignore_outliers.parquet")
results


# remove outliers from curves
# include old edgedroid
# choose the best models and carry this forward - experimentation, evaluation

Run:   0%|          | 0/40 [00:00<?, ?it/s]

2024-02-01 20:58:38.821 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: mean
2024-02-01 20:58:38.832 | INFO     | edgedroid.models.timings.curve:__init__:139 - Fitting execution time functions to data...
2024-02-01 20:58:38.861 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [-inf, 5.0): a * x^b + c (MSE: 0.03, prev. MSE 0.03)
2024-02-01 20:58:38.888 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [5.0, 9.0): a * x^b + c (MSE: 0.08, prev. MSE 0.08)
2024-02-01 20:58:38.892 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [9.0, inf): a * x^b + c (MSE: 0.07, prev. MSE 0.07)
2024-02-01 20:58:38.893 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: mean
2024-02-01 20:58:38.901 | INFO     | edgedroid.models.timings.curve:__init__:139 - Fitting execution time function

,real,prediction,error,sqr_error,model,duration,neuro
0,3.029,4.790592,1.761592,3.103207,1st order,"[-inf, 5.0)","[-inf, 0.5)"
1,3.029,5.261648,2.232648,4.984719,1st order (median),"[-inf, 5.0)","[-inf, 0.5)"
2,3.029,5.873496,2.844496,8.091159,1st order (mean),"[-inf, 5.0)","[-inf, 0.5)"
3,3.029,5.477779,2.448779,5.996520,a * x^b + c,"[-inf, 5.0)","[-inf, 0.5)"
4,3.029,5.247540,2.218540,4.921919,a * x^2 + b * x + c,"[-inf, 5.0)","[-inf, 0.5)"
...,...,...,...,...,...,...,...
54075,4.241,3.666877,-0.574123,0.329617,a * x^b + c,"[9.0, inf)","[0.5, inf)"
54076,4.241,3.651962,-0.589038,0.346965,a * x^2 + b * x + c,"[9.0, inf)","[0.5, inf)"
54077,4.241,3.236597,-1.004403,1.008825,a * x^3 + b * x^2 + c * x + d,"[9.0, inf)","[0.5, inf)"
54078,4.241,4.127838,-0.113162,0.012806,a * e^x + b,"[9.0, inf)","[0.5, inf)"


In [13]:
results.groupby(["model", "neuro", "duration"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

model                          neuro        duration   
multi curve                    [-inf, 0.5)  [9.0, inf)      2.868345
a * x^2 + b * x + c            [-inf, 0.5)  [9.0, inf)      2.935238
a * x^b + c                    [-inf, 0.5)  [9.0, inf)      2.935964
a * x^3 + b * x^2 + c * x + d  [-inf, 0.5)  [9.0, inf)      2.940189
a * e^x + b                    [-inf, 0.5)  [9.0, inf)      3.041867
multi curve                    [-inf, 0.5)  [5.0, 9.0)      3.371066
a * x^3 + b * x^2 + c * x + d  [-inf, 0.5)  [5.0, 9.0)      3.425255
a * e^x + b                    [-inf, 0.5)  [5.0, 9.0)      3.435473
a * x^b + c                    [-inf, 0.5)  [5.0, 9.0)      3.437181
a * x^2 + b * x + c            [-inf, 0.5)  [5.0, 9.0)      3.461366
multi curve                    [0.5, inf)   [5.0, 9.0)      3.585041
a * x^b + c                    [0.5, inf)   [5.0, 9.0)      3.646434
a * x^3 + b * x^2 + c * x + d  [0.5, inf)   [5.0, 9.0)      3.650062
a * e^x + b                    [0.5, inf)   [5.

In [14]:
results.groupby(["model"], observed=True)["sqr_error"].describe().sort_values(by="mean", ascending=True)

,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
multi curve,6083.0,3.667480,5.020740,1.863419e-09,0.417269,1.825393,5.162923,71.584008
a * x^3 + b * x^2 + c * x + d,6083.0,3.730183,5.110181,2.118783e-07,0.416468,1.857921,5.259032,72.562468
a * x^b + c,6083.0,3.744503,5.148141,5.012343e-07,0.464202,1.972060,5.135228,70.022594
a * x^2 + b * x + c,6083.0,3.752614,5.196787,1.742164e-07,0.466197,1.896098,5.141771,71.746707
a * e^x + b,6083.0,3.757297,5.108797,4.387432e-07,0.452675,1.987275,5.188291,69.163692
1st order (median),6083.0,3.969279,6.069358,6.192399e-08,0.456102,2.314209,4.775917,72.082642
1st order (mean),6083.0,4.489805,5.435193,1.201064e-07,0.809556,2.674773,6.238325,61.955413
1st order,6083.0,14.357629,67.277577,2.439705e-08,0.834634,3.931416,12.438491,2775.481508
